<a href="https://colab.research.google.com/github/PraveenMathew92/DatasetCreation/blob/main/DatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataSet Creation Assignment
CSE 576 - Natural Language Processing


## Explore the questions

In [42]:
from google.colab import files
files.upload()

{}

In [43]:
import json

qa_context_pairs = []
questions = []
FILE_NAME = 'train-v1.1.json'

with open(FILE_NAME) as f:
    data = json.load(f)
    title_data = data['data']
    for title_datum in title_data: 
        paragraphs = title_datum['paragraphs']
        for paragraph in paragraphs:
            context = paragraph['context']
            qas = paragraph['qas']
            qa_pairs = []
            for qa in qas:
                question = qa['question']
                questions.append(question)
                answer = qa['answers'][0]
                qa_pair = {}
                qa_pair['question'] = question
                qa_pair['answer'] = answer
                qa_pairs.append(qa_pair)
            qa_context_pair = {}
            qa_context_pair['context'] = context
            qa_context_pair['qas'] = qa_pairs
            qa_context_pairs.append(qa_context_pair)
            if len(qa_context_pairs) == 5000:
                break
        if len(qa_context_pairs) == 5000:
            break

In [44]:
questions = sorted(questions)
print(*questions, sep='\n')
print("\033[1m\033[91m COUNT =", len(questions), "\033[0m")

Streaming output truncated to the last 5000 lines.
When did the term "Near and Middle East" hold stage?
When did the term "classical music" appear? 
When did the term Nasara become used more in modern times?
When did the the prince of Rinbung occupy Lhasa?
When did the theater in the Plymouth Athenaeum close?
When did the torch arrive in Canberra?
When did the torch arrive in Dar es Salaam?
When did the torch arrive in Hong Kong?
When did the torch arrive in Islamabad?
When did the torch arrive in Muscat?
When did the torch arrive in Nagano?
When did the torch arrive in Pyongyang?
When did the torch arrive in Saint Petersburg?
When did the torch arrive in Vietnam?
When did the torch go through Bankok?
When did the torch reach Sanya, Hainan?
When did the torch relay in Paris occur?
When did the torch route begin in London?
When did the torch visit Macao?
When did the torch visit Malaysia?
When did the torch visit New Delhi?
When did the tour begin?
When did the tourism industry begin? 


### Extract Seed Questions

In [45]:
import random

SEED_SIZE = 100
random.seed(10)
seed_questions = random.sample(questions, SEED_SIZE)
sorted(seed_questions)

['A score-centric approach strictly emphasizes what?',
 'Along with high street chain stores and off-licenses, what stores undercut pub alcohol sales in the 1970s?',
 "Beyonce's grandma's name was?",
 'Bootylicious was a song from which act that Beyonce performed with?',
 'By 2009, by how much had the rate of HIV infection decreased?',
 'Children who have ADHD or conduct disorders respond better when what is a part of their treatment plan?',
 'Dogs often help clean in the kitchen by licking what?',
 'Dr. Babasaheb Ambedkar attempted to create a single combined document with all Buddhist principles and called it?',
 'How far was the relay route in kilometers in the Middle East?',
 'How many Episcopal churches were in the Boston Metro area in 2010?',
 'How many awards was Beyoncé nominated for at the 57th annual Grammys?',
 'How many departments are within the Stinson-Remick Hall of Engineering?',
 'How many people work in University of Oklahoma?',
 'How much money did it take to make Sp

In [46]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.1.0/en_core_web_trf-3.1.0.tar.gz

import spacy
nlp = spacy.load('en_core_web_trf')

     |████████████████████████████████| 460.2 MB 4.9 kB/s 


In [47]:
from transformers import pipeline

nlp_bert = pipeline("ner", grouped_entities=True)

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


In [48]:
from collections import Counter

def resolve_combined_entities(prev_entity, sequence):
    if len(prev_entity) == 1:
        return prev_entity[0]
    else:
        combined_score = sum([entity['score'] for entity in prev_entity]) / len(prev_entity)
        all_entities = [entity['entity_group'] for entity in prev_entity]
        occurence_count = Counter(all_entities)
        max_entity = occurence_count.most_common(1)[0][0]
        if combined_score > 0.5:
            return {
                'entity_group': max_entity,
                'score' : combined_score,
                'word' : sequence[prev_entity[0]['start']:prev_entity[-1]['end']],
                'start' : prev_entity[0]['start'],
                'end': prev_entity[-1]['end']

            }
        else:
            return None

In [49]:
BERT_ENTITIES = ['PER', 'ORG', 'LOC']

In [54]:
def get_named_entities(sequence):
    entities = nlp_bert(sequence)
    combined_entities = []
    prev_entity = []
    for entity in entities:
        if not prev_entity:
            if entity['score'] > 0.5:
                prev_entity.append(entity)
        elif prev_entity[-1]['end'] == entity['start'] :
            prev_entity.append(entity)
        else:
            actual_entity = resolve_combined_entities(prev_entity, sequence)
            if actual_entity:
                combined_entities.append(actual_entity)
            prev_entity = []
    if prev_entity:
      actual_entity = resolve_combined_entities(prev_entity, sequence)
      if actual_entity:
          combined_entities.append(actual_entity)
    
    entities = set()
    for combined_entity in combined_entities:
        is_required = False
        for default in BERT_ENTITIES:
            if combined_entity['entity_group'] in default:
                is_required = True
        if is_required:
            if 'PER' in combined_entity['entity_group']:
                final_entity = (combined_entity['start'], combined_entity['end'], 'PERSON')
                entities.add(final_entity)
            else:
                final_entity = (combined_entity['start'], combined_entity['end'], combined_entity['entity_group'])
                entities.add(final_entity)
    return entities

In [90]:
# try using lists in place of sets
def merge(spacy_ners, bert_ners):
  iter_spacy_ners = iter(spacy_ners)
  iter_bert_ners = iter(bert_ners)
  merged_ners = []
  try:
    for spacy_ner in iter_spacy_ners:
      merged_ners.append(spacy_ner)

      # add all bert models before the current spacy model
      bert_ner = next(iter_bert_ners)
      while spacy_ner[0] > bert_ner[1]:
        merged_ners.append(bert_ner)
        bert_ner = next(iter_bert_ners)
      
      # discard all bert models in the range of current spacy model
      while bert_ner[0] <= spacy_ner[1]:
        bert_ner = next(iter_bert_ners)
  except StopIteration:
    pass
  for spacy_ner in iter_spacy_ners:
    merged_ners.append(spacy_ner)
  for bert_ner in iter_bert_ners:
    merged_ners.append(bert_ner)
  print(spacy_ners, '\n', bert_ners, '\n', merged_ners, '\n')
  return merged_ners

In [91]:
import spacy

ner_list = []

for question in seed_questions:
  spacy_ners_for_question = set()
  for ent in nlp(question).ents:
    spacy_ners_for_question.add((ent.start_char, ent.end_char, ent.label_))

  bert_ners_for_question = get_named_entities(question)

  #combined_ners = spacy_ners_for_question | bert_ners_for_question
  #list_combined_ners = list(combined_ners)
  #list_combined_ners = sorted(list_combined_ners, key=lambda x: x[0])
  list_combined_ners = merge(sorted(spacy_ners_for_question, key=lambda x: x[0]), sorted(bert_ners_for_question, key=lambda x: x[0]))
  ner_list.append(list_combined_ners)

[] 
 [] 
 [] 

[(0, 12, 'WORK_OF_ART'), (44, 51, 'PERSON')] 
 [(44, 51, 'PERSON')] 
 [(0, 12, 'WORK_OF_ART'), (44, 51, 'PERSON')] 

[] 
 [] 
 [] 

[(61, 69, 'GPE'), (73, 82, 'DATE')] 
 [(61, 69, 'LOC')] 
 [(61, 69, 'GPE'), (73, 82, 'DATE')] 

[] 
 [] 
 [] 

[(96, 105, 'DATE')] 
 [] 
 [(96, 105, 'DATE')] 

[] 
 [] 
 [] 

[] 
 [] 
 [] 

[(59, 64, 'LANGUAGE')] 
 [] 
 [(59, 64, 'LANGUAGE')] 

[(9, 13, 'PERSON')] 
 [(9, 13, 'PERSON')] 
 [(9, 13, 'PERSON')] 

[(39, 42, 'NORP'), (43, 50, 'NORP')] 
 [] 
 [(39, 42, 'NORP'), (43, 50, 'NORP')] 

[(26, 30, 'PRODUCT')] 
 [] 
 [(26, 30, 'PRODUCT')] 

[(3, 7, 'DATE')] 
 [] 
 [(3, 7, 'DATE')] 

[(9, 21, 'DATE')] 
 [] 
 [(9, 21, 'DATE')] 

[(36, 46, 'CARDINAL'), (73, 79, 'ORG'), (84, 91, 'ORG'), (95, 99, 'DATE')] 
 [(73, 91, 'ORG')] 
 [(36, 46, 'CARDINAL'), (73, 79, 'ORG'), (84, 91, 'ORG'), (95, 99, 'DATE')] 

[(8, 23, 'PERSON'), (46, 61, 'PERSON')] 
 [(8, 21, 'PERSON')] 
 [(8, 23, 'PERSON'), (46, 61, 'PERSON')] 

[(20, 27, 'PERSON'), (49, 53, 'ORDINAL

In [97]:
EXCLUDED_NERS = {'CARDINAL', 'ORDINAL'}

for index, question in enumerate(seed_questions):
  question_copy = str(question)
  question_ner_list = ner_list[index]
  for question_ner in reversed(list(question_ner_list)):
    start, end, ner = question_ner
    if ner in EXCLUDED_NERS:
      continue
    # question_copy = question_copy[:start] + '-'*(end - start) + question_copy[end:]
    question_copy = question_copy[:start] + '-' + question_copy[end:]
  print("\033[1m", question , "\033[0m")
  print("\t", question_copy, end='\n')

 Where was the worst of the quake lakes located? 
	 Where was the worst of the quake lakes located?
 Bootylicious was a song from which act that Beyonce performed with? 
	 - was a song from which act that - performed with?
 What seperates the children in secondary school? 
	 What seperates the children in secondary school?
 What was the name of the new musical style that emerged from New York in the 1970s? 
	 What was the name of the new musical style that emerged from - in -?
 Which athlete did the official website call an angel? 
	 Which athlete did the official website call an angel?
 Along with high street chain stores and off-licenses, what stores undercut pub alcohol sales in the 1970s? 
	 Along with high street chain stores and off-licenses, what stores undercut pub alcohol sales in -?
 The cycle of rebirth is also called what? 
	 The cycle of rebirth is also called what?
 What was a function of distant country pubs before the rise of motor vehicles? 
	 What was a function of di

In [58]:
from spacy import displacy
question = "What is the name of Portugal's central bank?"
displacy.render(nlp(question), style="ent", jupyter=True)
#for question in seed_questions:
  

## List the questions

In [ ]:
for paragraphs in json_blob['data']:
  print(paragraphs)
  for paragraph in paragraphs:
    print(paragraph)
    for qa in paragraph['paragraphs']:
      print(qa['question'])

In [ ]:
[x['question'] for x in json_blob['data'][0]['paragraphs'][0]['qas']]

In [ ]:
for paragaphs in json_blob['data'][0]['paragraphs']:
  